Load API Keys

In [41]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [2]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n      

Split the Loaded Data into Chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(data)
texts

[Document(page_content='Fixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n                            \n\nDeposits\nCredit Card\nLoans\nInsurance\nInvestment\nRemittance\

Initialize Pinecone

In [4]:
import os
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit-with-conversation-chain-gpt3.5"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

e:\Projects\Langchain\venv\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


{}

Create Embeddings and Vectors to Insert to Pinecone Database

In [5]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = Pinecone.from_documents(texts, embeddings, index_name=index_name, namespace=namespace)

Creating Prompt Template

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: {question}
  By only basing on this context: {context}
  If the question does not specify the fixed deposit type, answer the question assuming it is about fixed deposit account.
  If you don't have enought information from the document, don't answer with your general knowledge, just say you don't have that information.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the question, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input question you to do something creative such as writing a song or making a joke.
"""
prompt = PromptTemplate.from_template(prompt_template)

Create Retriever for Vector Search

In [7]:
retriever = vector.as_retriever()
retriever

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x00000185FF3C7450>)

Create MultiQuery retriever

In [8]:
from langchain.retrievers.multi_query import MultiQueryRetriever
import logging

multi_query_retriever = MultiQueryRetriever.from_llm(
  llm=llm,
  retriever=retriever
)

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [9]:
multi_query_retriever.get_relevant_documents("How many types of fixed deposit does HongLeong Bank provide?")

INFO:langchain.retrievers.multi_query:Generated queries: ['What are the different types of fixed deposit offered by HongLeong Bank?', 'What options for fixed deposit are available at HongLeong Bank?', 'What are the various fixed deposit choices provided by HongLeong Bank?']


[Document(page_content='Fixed Deposit | Hong Leong Bank Malaysia\n\n\n\n\n\n\n\n\n\n\nFixed Deposit\n \nnull\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n                            \n\nDeposits\nCredit Card\nLoans\nInsurance\nIn

Contextual Compression

In [62]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline, EmbeddingsFilter
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
  splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0, separator=". ")
  llm_extractor_compressor = LLMChainExtractor.from_llm(llm)
  # redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
  # relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.8)
  # pipeline_compressor = DocumentCompressorPipeline(
  #   transformers=[splitter, llm_extractor_compressor, redundant_filter, relevant_filter]
  # )
  compression_retriever = ContextualCompressionRetriever(
      base_compressor=llm_extractor_compressor, base_retriever=multi_query_retriever
  )
  result = compression_retriever.get_relevant_documents("How many types of fixed deposit does HongLeong Bank provide?")
  print(f'{cb}\n')
  print(result)

INFO:langchain.retrievers.multi_query:Generated queries: ['What are the different types of fixed deposit offered by HongLeong Bank?', 'What options for fixed deposit are available at HongLeong Bank?', 'What is the variety of fixed deposit choices provided by HongLeong Bank?']
e:\Projects\Langchain\venv\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
e:\Projects\Langchain\venv\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
e:\Projects\Langchain\venv\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
e:\Projects\Langchain\venv\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is depr

Tokens Used: 1736
	Prompt Tokens: 1619
	Completion Tokens: 117
Successful Requests: 7
Total Cost (USD): $0.001853

[Document(page_content='Deposits', metadata={'description': 'Hong Leong Bank pays the highest interest on fixed deposit. Let your child earn with HLB Junior Fixed Deposit, a rewarding fixed deposit account for children.', 'language': 'en', 'source': 'https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html', 'title': 'Hong Leong Bank - Junior Fixed Deposit'}), Document(page_content='Junior Fixed Deposit account is designed for children below the age of 18. It gives attractive\xa0interest that’s calculated daily. Fixed deposit placement starts from RM1,000.', metadata={'description': 'Hong Leong Bank pays the highest interest on fixed deposit. Let your child earn with HLB Junior Fixed Deposit, a rewarding fixed deposit account for children.', 'language': 'en', 'source': 'https://www.hlb.com.my/en/personal-banking/fixed-deposit

Creating Memory

In [59]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm, memory_key="chat_history", output_key="answer", max_token_limit="500", return_messages=True)

Creating Chain

In [63]:
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks import get_openai_callback

chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=compression_retriever,
  memory=memory,
  combine_docs_chain_kwargs={"prompt": prompt},
  verbose=True,
  return_source_documents=True
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain(question)
    print(f'{cb}\n')
    print(f'\nAnswer: {result["answer"]}')
    for source_document in result["source_documents"]:
      print(f'\Documents: {source_document}')


Running the Chain

In [51]:
ask_question("What is fixed deposit?")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you explain the concept of fixed deposit?', '2. How would you define fixed deposit?', '3. What are the key characteristics of a fixed deposit?']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: What is fixed deposit?
  By only basing on this context: It gives attractive interest that’s calculated daily. Fixed deposit placement starts from RM1,000.

Junior Fixed Deposit






More Details



An attractive interest FD account for children below the age of 18










Foreign Currency Fixed Deposit






More Details



A convenient way to invest in foreign currency




































Credit Card Name




















Our Artificial Intelligence Advisor is ready to assist you






Chat with us







×









×  
External Link Disclaimer


You are leaving Hong Leong Bank's website as such our Privacy Notice shall cease

Fixed Deposit - HLB










Fixed Deposit - HLB
 























PERSONAL


BUSINESS



     

In [64]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: HongLeong Bank provides three types of fixed deposit accounts: eFixed Deposit, Junior Fixed Deposit, and Senior Savers Flexi Fixed Deposit. The eFixed Deposit account is a convenient and flexible option for customers to manage their fixed deposit online. The Junior Fixed Deposit account is specifically designed for children below the age of 18, encouraging them to start saving at a young age. The Senior Savers Flexi Fixed Deposit account is tailored for individuals aged 50 years and above, offering them flexibility and attractive returns on their savings.
Follow Up Input: How many types of fixed deposit does HongLeong Bank provide?
Standalone question:

> Finished chain.


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the various fixed deposit account types offered by HongLeong Bank?', '2. What are the specific categories or options for fixed deposit accounts available at HongLeong Bank?', '3. Could you please outline the different types of fixed deposit accounts that customers can open at HongLeong Bank?']
e:\Projects\Langchain\venv\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
e:\Projects\Langchain\venv\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
e:\Projects\Langchain\venv\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
e:\Projects\



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: What are the three types of fixed deposit accounts provided by HongLeong Bank?
  By only basing on this context: Deposits

Junior Fixed Deposit account is designed for children below the age of 18. It gives attractive interest that’s calculated daily. Fixed deposit placement starts from RM1,000.

Senior Savers Flexi Fixed Deposit account is designed for individuals aged 50 years and above. It gives attractive interest and allows partial withdrawal anytime without losing your FD interest on account balance.
  If the question does not specify the fixed deposit type, answer the question assuming it is about fixed deposit account.
  If you don't have enought information from the document, don't answer with your general knowledge, just say you don't have th

In [ ]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")

In [ ]:
ask_question("What are the interest rates for Fixed Deposit?")

In [ ]:
ask_question("Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw  at the end of the term.")